In [2]:
from vrp_viz import VRPPlot
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
import time

# Folium

In [15]:
coords = [(48.20817, 16.37382),
          (46.987536362460084, 7.427499610340133),
          (41.86933058455828, 12.662484881620202),
          (50.07589292509622, 14.435404050834023),
          (52.509346514796945, 5.323617705412682),
          (40.727924387874644, -3.781408384468865),
          (43.01009091777951, 23.23397235157129),
          (48.20817, 16.37382)]
trails = [[0, 1, 2, 0],
          [0, 3, 4, 0],
          [0, 5, 6, 0]]

In [17]:
VRPPlot(coords=coords,
        trails=trails) \
        .show()

# Geopy

In [3]:
app = Nominatim(user_agent="JournalDev")

In [4]:
def _prepare_data(address_list):

    df = pd.DataFrame({"lat": [],
                       "lon": []})

    for address in address_list:
      try:
        time.sleep(1)
        location = app.geocode(address).raw
        df.loc[len(df.index)] = [location['lat'], location['lon']]
      except Exception as e:
        print(e)

    return [(float(df.loc[index, 'lat']), float(df.loc[index, 'lon']))
            for index, row in df.iterrows()]

In [5]:
def _create_data_model(distances):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distances
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [6]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} km'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}km\n'.format(route_distance)

In [7]:
def return_solution(manager, routing, solution):
    index = routing.Start(0)
    output = []
    while not routing.IsEnd(index):
        output.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    output.append(manager.IndexToNode(index))
    return output

In [8]:
def perform_TSP(data, manager, routing):
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Return solution or print solution on console
    if solution:
        return solution


# Zamki w Polsce

In [9]:
address_list = ['Wawel Castle in Krakow, Poland',
                'Malbork Castle, Poland',
                'Warsaw Royal Castle, Poland',
                'Książ Castle, Poland',
                'Czocha Castle, Poland',
                'Niedzica Castle, Poland']

locations = _prepare_data(address_list=address_list)
distances = [[int(geodesic(a,b).km) for a in locations]
                          for b in locations]

data = _create_data_model(distances=distances)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

solution = perform_TSP(data=data, manager=manager, routing=routing)

In [10]:
print_solution(manager=manager, routing=routing, solution=solution)

Objective: 1388 km
Route for vehicle 0:
 0 -> 5 -> 3 -> 4 -> 1 -> 2 -> 0



In [11]:
locations.append(locations[0])
route = return_solution(manager=manager, routing=routing, solution=solution)
VRPPlot(coords=locations,
        trails=[route]) \
        .show()

# Europejskie stolice

In [26]:
capitals = ['Zurich',
            'Vienna',
            'Rome',
            'Prague',
            'Amsterdam',
            'Madrid',
            'Sofia, Bulgaria']

locations = _prepare_data(address_list=capitals)
distances = [[int(geodesic(a,b).km) for a in locations]
                          for b in locations]

data = _create_data_model(distances=distances)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

solution = perform_TSP(data=data, manager=manager, routing=routing)

In [27]:
print_solution(manager=manager, routing=routing, solution=solution)

Objective: 5906 km
Route for vehicle 0:
 0 -> 5 -> 2 -> 6 -> 1 -> 3 -> 4 -> 0



In [28]:
locations.append(locations[0])
route = return_solution(manager=manager, routing=routing, solution=solution)
VRPPlot(coords=locations,
        trails=[route]) \
        .show()